In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{B}i$ for $i\in\{0,\cdots,n\}$ is the number of boats on the western shore after the 
  $i^{\textrm{th}}$ crossing.

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [ ]:
function flatten<T>(LoL: T[][]): T[] {
    return LoL.reduce((acc, val) => acc.concat(val), []);
}

In [ ]:
flatten([[1,2], [3,4]]);

The function `start` takes three numbers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `true` in the initial state where everybody is on the western shore.

In [ ]:
function start(M: number, C: number, B: number): boolean {
    return M === 3 && C === 3 && B === 1;
}
(globalThis as any).start = start;

The function `goal` takes three numbers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `true` in the state where everybody is on the eastern shore
and hence nobody is left on the western shore.

In [ ]:
function goal(M: number, C: number, B: number): boolean {
    return M === 0 && C === 0 && B === 0;
}
(globalThis as any).goal = goal;

The function `invariant` takes three numbers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `true` if there is no problem on either shore of the river.  
There is no problem if any one of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. 
  $\texttt{M} = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M} = 3$.
  Then there are no missionaries on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of 
  infidels on that side, i.e. $\texttt{M} = \texttt{C}$.  Then the numbers of 
  missionaries and infidels have to match on the eastern shore as well.
  
Furthermore, as there is just one boat, the number $\texttt{B}$ of boats is at most one.

In [ ]:
function invariant(M: number, C: number, B: number): boolean {
    return (M === 0 || M === 3 || M === C) && B <= 1;
}
(globalThis as any).invariant = invariant;

The function `transition` takes 6 arguments:
* `M𝛼` is the number of missionaries on the western shore before the crossing.
* `C𝛼` is the number of infidels on the western shore before the crossing.
* `B𝛼` is the number of boats on the western shore before the crossing. 
* `M𝛽` is the number of missionaries on the western shore after the crossing.
* `C𝛽` is the number of infidels on the western shore after the crossing.
* `B𝛽` is the number of infidels on the western shore after the crossing.
  

The function call `transition(M𝛼, C𝛼, B𝛼, M𝛽, C𝛽, B𝛽)` returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the crossing.  Note that if `B𝛼 == 1`, then the boat travels from the western shore to the eastern shore.  If `B𝛼 == 0`, the boat travels from the eastern shore to the western shore.

The condition 
```
   1 <= M𝛼 - M𝛽 + C𝛼 - C𝛽 <= 2
```
ensures that the boat has at least one and at most two passengers when travelling from west to east.  The conditions
```
M𝛽 <= M𝛼 and C𝛽 <= C𝛼
```
ensures that the number of missionaries and infidels on the western shore does not increase when the boat travels from the western shore to the eastern shore.

In [ ]:
function transition(Ma: number, Ca: number, Ba: number, Mb: number, Cb: number, Bb: number): boolean {
    if (Bb !== 1 - Ba) return false;
    if (Ba === 1) { 
        return (1 <= (Ma - Mb) + (Ca - Cb) && (Ma - Mb) + (Ca - Cb) <= 2 && Mb <= Ma && Cb <= Ca);
    } else {
        return (1 <= (Mb - Ma) + (Cb - Ca) && (Mb - Ma) + (Cb - Ca) <= 2 && Mb >= Ma && Cb >= Ca);
    }
}
(globalThis as any).transition = transition;

In [ ]:
import * as BCS from "./02-Backtracking-Constraint-Solver";

In [ ]:
type MissionariesCSP = BCS.CSP<string, number>;

The function `missionariesCSP` creates a CSP that tries to solve the problem with `n` crossings.

In [ ]:
function missionariesCSP(n: number): MissionariesCSP {
    const lists: string[][] = [];
    for (let i = 0; i <= n; i++) {
        lists.push([`M${i}`, `C${i}`, `B${i}`]);
    }
    const variables = flatten(lists); 
    const values = [0, 1, 2, 3];
    
    const constraints: string[] = [];
    constraints.push(`start(M0, C0, B0)`);
    constraints.push(`goal(M${n}, C${n}, B${n})`);
    
    for (let i = 0; i < n; i++) {
        constraints.push(`invariant(M${i}, C${i}, B${i})`);
        constraints.push(`transition(M${i}, C${i}, B${i}, M${i+1}, C${i+1}, B${i+1})`);
    }
    
    return {
        variables,
        values,
        constraints
    };
}

In [ ]:
missionariesCSP(3);

The function `findSolution` computes a solution to the problem of saving the infidels.

In [ ]:
function findSolution(): [number, BCS.Solution<string, number>] {
    let n = 1;
    while (true) {
        console.log(`Checking n=${n}...`);
        const csp = missionariesCSP(n);
        const result = BCS.solve(csp);
        
        if (result !== null) {
            return [n, result];
        }
        n += 2;
    }
}

On my desktop computer (2023 MacStudio with M2 Max processor) it takes about 1 second to solve the problem. 

In [ ]:
console.time("findSolution"); 
const result = findSolution(); 
console.timeEnd("findSolution");
result

In [ ]:
function showSolution(solution: BCS.Solution<string, number>, n: number) {
    for (let i = 0; i <= n; i++) {
        const M = Number(solution[`M${i}`]);
        const C = Number(solution[`C${i}`]);
        const B = Number(solution[`B${i}`]);
        
        const leftSide = '😇'.repeat(M) + '🥷'.repeat(C);
        const rightSide = '😇'.repeat(3 - M) + '🥷'.repeat(3 - C);
        const gap = " ".repeat(28);
        
        console.log(`${leftSide}${gap}${rightSide}`);
        
        if (i < n) {
            const nextM = Number(solution[`M${i+1}`]);
            const nextC = Number(solution[`C${i+1}`]);
            
            if (B === 1) {
                const MB = M - nextM;
                const CB = C - nextC;
                console.log(' '.repeat(12) + `>>> ${'😇'.repeat(MB)}${'🥷'.repeat(CB)} >>>`);
            } else {
                const MB = nextM - M;
                const CB = nextC - C;
                console.log(' '.repeat(12) + `<<< ${'😇'.repeat(MB)}${'🥷'.repeat(CB)} <<<`);
            }
        }
    }
}

In [ ]:
const [n, solution] = result;
showSolution(solution, n);